In [4]:
# This code is run in Google Collab - easiest environment to run these quantum simulations
from numpy.lib.function_base import kaiser
from pytket.utils import probs_from_counts, counts_from_shot_table, expectation_from_counts

import numpy as np

# We use this paper to reconstruct our density matrix by IBM: https://arxiv.org/pdf/1106.5458.pdf
# We will do the fidelity reconstruction over multiple qubits to allow generalization

I = np.array([[1, 0], [0, 1]])
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])
basis = [I, X, Y, Z]

# Direct computation of expectation using numpy operations
def get_expectation(shot_table):
  aritysum = np.sum(np.sum(shot_table, axis=1) % 2)
  return (-2 * aritysum) / shot_table.shape[0] + 1

# Use pytket library methods to compute expectation from shot table
def pytket_get_expectation(shot_table):
  return expectation_from_counts(counts_from_shot_table(shot_table))

# Compute the tensor product of all the basis elements from the given index
# The bits are grouped into blocks of two which are used to identify which element
# of the Pauli matrices would be used to build the projective measurement operator
def get_n_qubit_basis_element(index, nqubits):
  n_qubit_basis_element = np.array([1.])
  for qubit_index in range(nqubits):
    # Use a bit mask to retrieve the corresponding block of 2 bits that defines the index
    mask = 3 << (2 * qubit_index)
    basis_index = (index & mask) >> (2 * qubit_index)

    # np.kron computes the tensor product of the given matrices
    n_qubit_basis_element = np.kron(n_qubit_basis_element, basis[basis_index])
  return n_qubit_basis_element

# This builds the Maximum Likelihood Estimation (MLE) of the density matrix from the
# expectation of our shot data. Each projective measurement would be weighted by
# its expectation in the sum that we are returning.
def build_mle_matrix(expectations, nqubits):
  dim = 2 ** nqubits
  result = np.zeros((dim, dim))
  for index, expectation in enumerate(expectations):
    result = result + expectation * get_n_qubit_basis_element(index, nqubits)
  return result

# Construct the density matrix given shot data for d^2 Hermitian projectors that span the dxd matrix space
# d = 2^n is the dimension of one side of the matrix
def get_density_matrix(shot_tables, nqubits):
  dim = 2 ** nqubits
  # We use the shot data to construct expectations - we assume that they follow the order in which we
  # construct our matrices that represent some projective measurement
  expectations = np.array([pytket_get_expectation(shot_table) for shot_table in shot_tables])
  mle_matrix = build_mle_matrix(expectations, nqubits) / dim

  # We obtain the eigenvalues and eigenvectors and sort them from largest to smallest
  # We maintain a mapping of the associated eigenvector which they belong to
  eigenvalues, eigenvectors = np.linalg.eig(mle_matrix)
  mapped_eigenvalues = [(eigenvalue, i) for i, eigenvalue in enumerate(eigenvalues)]
  mapped_eigenvalues.sort()
  mapped_eigenvalues.reverse()

  # From the IBM Paper, they computed an explicit solution for the least squares regression
  # of a positive definite matrix to the MLE solution. The main problem is that the MLE
  # solution can have negative eigenvalues which is illegal in a density matrix. We want a
  # positive matrix which by definition only has nonnegative eigenvalues.
  accumulator = 0
  indicator = -1
  for i in range(len(mapped_eigenvalues)-1, -1, -1):
    eigenvalue, eigenvector_index = mapped_eigenvalues[i]
    k = i + 1
    indicator = eigenvalue + accumulator / k
    mapped_eigenvalues[i] = (max(0, indicator), eigenvector_index)
    if indicator < 0:
      accumulator += eigenvalue

  print("Reduced Eigenvalues: ", mapped_eigenvalues)

  # Build the solution with the recalculated eigenvalues
  density_matrix = np.zeros((dim, dim), dtype=np.complex128)
  for eigenvalue, index in mapped_eigenvalues:
    eigenvector = eigenvectors[index].reshape((-1, 1))
    density_matrix += (eigenvector @ np.matrix.getH(eigenvector)) * eigenvalue
  return density_matrix

# Compute squareroot of matrix using diagonalization and square root of eigenvalues
def get_sqrt_matrix(density_matrix):
  evalues, evectors = np.linalg.eig(density_matrix)
  sqrt_matrix = (evectors * np.sqrt(evalues)) @ np.linalg.inv(evectors)
  return sqrt_matrix

# We now compute fidelity of the circuit given the shot_tables and the expected density matrix
def get_fidelity(expected_density_matrix, shot_tables, nqubits):
  density_matrix = get_density_matrix(shot_tables, nqubits)
  sqrt_matrix = get_sqrt_matrix(density_matrix)
  return np.trace(get_sqrt_matrix(sqrt_matrix @ expected_density_matrix @ sqrt_matrix)) ** 2



In [5]:
from pytket import Circuit
from pytket.extensions.qiskit import AerBackend

from pytket import Circuit
from pytket.circuit.display import get_circuit_renderer

circuit_renderer = get_circuit_renderer() # Instantiate a circuit renderer
circuit_renderer.set_render_options(zx_style=True) # Configure render options
circuit_renderer.condense_c_bits = False # You can also set the properties on the instance directly
print("Render options:")
print(circuit_renderer.get_render_options()) # View currently set render options

circuit_renderer.min_height = "300px" # Change the display height

# We obtain a list of shot tables over the basis of projective measurements
# Circuit should not have performed measurement of ANY kind
def obtain_shots_information(circuit, nqubits, backend, nshots):
  matrix_basis_dim = 4 ** nqubits
  shots_tables = []
  # Similar to the tensor product construction of the projective measurement matrix
  # We use a basis index to identify which measurements should be done for each qubit
  for basis_index in range(matrix_basis_dim):
    base = circuit.copy()
    for qubit_index in range(nqubits):
      # Obtain which basis to measure over a single qubit using a bitmask
      mask = 3 << (2 * qubit_index)
      operator_index = (basis_index & mask) >> (2 * qubit_index)

      # I (index 0) - Discard this qubit in this particular measurement
      # X (index 1) - Add a Hadamard gate to measure in X basis
      # Y (index 2) - Add a S dagger and Hadamard to measure in Y basis
      # Z (index 3) - Do Nothing - Measurement performed in Z basis
      if operator_index == 0:
        continue

      if operator_index == 1:
        base.H(qubit_index)

      if operator_index == 2:
        base.Sdg(qubit_index)
        base.H(qubit_index)

      base.Measure(qubit_index, qubit_index)

    compiled_circ = backend.get_compiled_circuit(base)
    handle = backend.process_circuit(compiled_circ, n_shots=nshots)
    shots = backend.get_result(handle).get_shots()
    shots_tables.append(shots)
  return shots_tables

Render options:
{'zx_style': True, 'condense_c_bits': False}


In [7]:
from pytket import Circuit
from pytket.extensions.qiskit import AerBackend

nqubits=3
nshots=10000
circ = Circuit(nqubits, nqubits)
circ.H(0).H(1).X(1).CX(1, 2)
backend = AerBackend()

shots_tables = obtain_shots_information(circ, nqubits, backend, nshots)
matrix = get_density_matrix(shots_tables, nqubits)
print(matrix)

Reduced Eigenvalues:  [((0.9816007856818981+1.2330751205340527e-17j), 0), ((0.0021574729453281358-1.4178672911017318e-18j), 2), ((0.0008328295629981778-2.335837003577415e-18j), 5), (0, 6), (0, 7), (0, 4), (0, 3), (0, 1)]
[[ 0.24115187+3.02928822e-18j  0.05127965-5.06754876e-02j
  -0.04980205+5.60338365e-02j -0.11784838+1.66988998e-01j
  -0.08296409-5.61272241e-02j  0.21937599+1.34327911e-03j
  -0.24820735+4.97998608e-02j -0.03306368+5.12674907e-02j]
 [ 0.05127965+5.06754876e-02j  0.02225883-3.27181933e-19j
  -0.02232074+1.36903867e-03j -0.06014677+1.05720179e-02j
  -0.00607147-2.94422860e-02j  0.04684298+4.63213280e-02j
  -0.06261714-4.15509086e-02j -0.01802214+4.37215958e-03j]
 [-0.04980205-5.60338365e-02j -0.02232074-1.36903867e-03j
   0.02335626+1.98260601e-19j  0.06313941-7.09998608e-03j
   0.00393423+3.08911956e-02j -0.04498342-5.11390633e-02j
   0.06290774+4.74601810e-02j  0.01863357-2.87098650e-03j]
 [-0.11784838-1.66988998e-01j -0.06014677-1.05720179e-02j
   0.06313941+7.099986